# Image Classifier



---


Rede CNN com 4 camadas pra classificação de imagens.

Para versões do Keras >= 2 não utilizar padding='same' para strides != 1. De acordo com a documentação do Keras é incompatível. [ver aqui](https://keras.io/layers/convolutional/)

---

Para treinamento com classes binárias alterar os seguintes parâmetros:

* loss='categorical_crossentropy' para loss='binary_crossentropy'
* class_mode="categorical" para class_mode="binary"
* Na última "Dense" alterar o número de valores de saída pra "1", se for categorical deve ser o número de categorias possíveis



### Verificando configurações do TensorFlow

In [ ]:
# Checando a versão do Tensorflow
import tensorflow
tensorflow.__version__

In [ ]:
# Verificando o funcionamento da GPU
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

### Instando bibliotecas faltantes (uso do colab)

In [ ]:
# Instalando biblioteca para plotar gráfico durante o treinamento
!pip install livelossplot
from livelossplot import PlotLossesKeras

### Variáveis para outup do treino

In [ ]:
# Declarando variáveis para registros de saída
MODEL_SUMMARY_FILE = "model_summary_v1.txt"
TRAINING_LOGS_FILE = "training_logs_v1.csv"
MODEL_FILE = "model_three_categories_v1.h5"

### Importando bibliotecas

In [ ]:
# Importando bibiotecas necessárias
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import CSVLogger
from livelossplot import PlotLossesKeras
from google.colab import drive, files
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn import metrics
from keras.models import load_model
from keras.callbacks import EarlyStopping
from mlxtend.plotting import plot_confusion_matrix

### Definindo path a partir do Google Drive

In [ ]:
path = ""
drive.mount('/content/drive/')
path = "/content/drive/My Drive/image_classifier/"
path_to_save_model = path + "save_model/" + MODEL_FILE 
training_data_dir = path + "dataset/train"
validation_data_dir = path + "dataset/val"
test_data_dir = path + "dataset/test"

### Hyperparâmetros

In [ ]:
IMAGE_WIDTH, IMAGE_HEIGHT = 200, 200
EPOCHS = 100
BATCH_SIZE = 32
TEST_SIZE = 20

input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, 3)

### Estrutura da Rede

In [ ]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=3, strides=1, input_shape=input_shape, padding='same', activation='relu'))
model.add(Conv2D(filters=32, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_last'))

model.add(Conv2D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(Conv2D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_last'))

model.add(Conv2D(filters=128, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(Conv2D(filters=128, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_last'))

model.add(Conv2D(filters=256, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(Conv2D(filters=256, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_last'))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(3))
model.add(Activation('sigmoid'))
    
model.compile(loss='categorical_crossentropy',
            optimizer=RMSprop(lr=0.0001),
            metrics=['accuracy'])

summary_path = path + MODEL_SUMMARY_FILE
with open(summary_path,"w") as fh:
    model.summary(print_fn=lambda line: fh.write(line + "\n"))

### Preparando Dados

In [ ]:
# Aumentando Dados
training_data_generator = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True)
validation_data_generator = ImageDataGenerator(rescale=1./255)
test_data_generator = ImageDataGenerator(rescale=1./255)

# Preparando Dados
training_generator = training_data_generator.flow_from_directory(
    training_data_dir,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode="categorical")
validation_generator = validation_data_generator.flow_from_directory(
    validation_data_dir,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode="categorical")
test_generator = test_data_generator.flow_from_directory(
    test_data_dir,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=1,
    class_mode="categorical", 
    shuffle=False)

### Treinando

In [ ]:
# Configurando o Callbak para salvar quando o loss parar de diminuir
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Criando Path para salvar no diretório do 
training_logs_path = path + TRAINING_LOGS_FILE

# Treinamento
model.fit_generator(
    training_generator,
    steps_per_epoch=len(training_generator.filenames) // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=len(validation_generator.filenames) // BATCH_SIZE,
    callbacks=[PlotLossesKeras(), CSVLogger(training_logs_path,
                                            append=False,
                                            separator=","), early_stopping], 
    verbose=1)

# salvando modelo
model.save(path_to_save_model)

### Testando

In [ ]:
# Carregando Dados de Teste

# Veririca a quantidade de classes e retorna y_predict
def probas_to_classes(y_pred):
    if len(y_pred.shape) > 1 and y_pred.shape[1] > 1:
        return categorical_probas_to_classes(y_pred)
    return np.array([1 if p > 0.5 else 0 for p in y_pred])
  
def categorical_probas_to_classes(p):
    return np.argmax(p, axis=1)
  
# -------------------------------------------

# Obtem o nome de todos os arquivos de teste
filenames = test_generator.filenames

# Obtem o total de arquivos de teste
nb_samples = len(filenames)

# Retorna os valores percentuais preditos pelo modelo para cada classe
predict = model.predict_generator(test_generator, steps=nb_samples)

# Retorna a maior valor predito
y_pred = probas_to_classes(predict)

# Retorna o valor real para cada classe
y_true = test_generator.classes



### Imprimindo a predição para cada imagem de teste


In [ ]:
# Imprime o caminho da imagens o valor real e a predição
for ele in list(zip(filenames, y_true, y_pred)):
    print(ele)
    
# Imprime a accuracy final do modelo
accuracy = accuracy_score(y_true, y_pred)

### Matriz de Confusão

In [ ]:
class_names = list(test_generator.class_indices.keys())
c_matrix = confusion_matrix(y_true, y_pred)
classes = [''] + class_names

fig, ax = plot_confusion_matrix(conf_mat=c_matrix,
                                colorbar=True,
                                figsize=(8,8),
                                cmap='GnBu',
                                show_absolute=False,
                                show_normed=True)


ax.xaxis.set_ticklabels(classes, fontsize=12); 
ax.yaxis.set_ticklabels(classes, fontsize=12);
ax.set_title('Confusion Matrix', fontsize=16);


plt.show();